In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q farm-haystack[colab,inference,metrics,elasticsearch,preprocessing]
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.2/768.2 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... don

create new test set

In [ ]:
import os
import json
import random

data_path = '/content/drive/MyDrive/extra_translations_DPR'
all_data = []

for each in os.listdir(data_path):
  dev_path = data_path + '/' + each + '/' + each + '_dev_SQuAD_format.json'

  with open(dev_path, 'r', encoding = 'utf-8-sig') as f:
    dev_data = json.load(f)

  all_data += dev_data['data'][:65]


random.shuffle(all_data)

data_dict = {}

data_dict['data'] = all_data

with open('/content/drive/MyDrive/new_dev_SQuAD_format.json', 'w') as fout:
  json.dump(data_dict, fout, ensure_ascii = False)

# Retrieval Evaluation

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
from haystack.utils import launch_es
import time

launch_es()
time.sleep(30)

In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

doc_index = "hybrid_docs"
label_index = "hybrid_labels"

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index=doc_index,
    label_index=label_index,
    embedding_field="emb",
    embedding_dim=768,
    excluded_meta_data=["emb"],
)

In [ ]:
from haystack.nodes import PreProcessor

preprocessor = PreProcessor(
    split_by="word",
    language = 'fa',
    split_length=200,
    split_overlap=0,
    split_respect_sentence_boundary=False,
    clean_empty_lines=False,
    clean_whitespace=False,
)
document_store.delete_documents(index=doc_index)
document_store.delete_documents(index=label_index)

# The add_eval_data() method converts the given dataset in json format into Haystack document and label objects. Those objects are then indexed in their respective document and label index in the document store. The method can be used with any dataset in SQuAD format.

# docs_to_index = preprocessor.process(documents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
document_store.delete_documents()
# document_store.write_documents(docs_to_index)
document_store.add_eval_data(
    filename="/content/drive/MyDrive/new_dev_SQuAD_format.json",
    doc_index=doc_index,
    label_index=label_index,
    preprocessor=preprocessor,
)

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 2554.39docs/s]


In [ ]:
from haystack.nodes import DensePassageRetriever, BM25Retriever

sparse_retriever = BM25Retriever(document_store=document_store)
dense_retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="/content/drive/MyDrive/saved_models/dpr_parsbert/query_encoder",
                                  passage_embedding_model="/content/drive/MyDrive/saved_models/dpr_parsbert/passage_encoder",
                                  use_gpu=True,
                                  max_seq_len_passage=256,
                                  embed_title=True)


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
document_store.update_embeddings(retriever=dense_retriever)

Create embeddings: 100%|██████████| 704/704 [00:10<00:00, 89.20 Docs/s]
Updating embeddings: 10000 Docs [00:16, 618.04 Docs/s]


In [ ]:
from haystack.nodes import JoinDocuments, SentenceTransformersRanker

join_documents = JoinDocuments(join_mode="concatenate")
# rerank = SentenceTransformersRanker(model_name_or_path="cross-encoder/ms-marco-MiniLM-L-6-v2")
rerank = SentenceTransformersRanker(model_name_or_path="/content/drive/MyDrive/cross_encoder/cross_encoder-2024-02-10_10-21-21")

In [ ]:
from haystack.pipelines import Pipeline

pipeline = Pipeline()
pipeline.add_node(component=sparse_retriever, name="SparseRetriever", inputs=["Query"])
pipeline.add_node(component=dense_retriever, name="DenseRetriever", inputs=["Query"])
pipeline.add_node(component=join_documents, name="JoinDocuments", inputs=["SparseRetriever", "DenseRetriever"])
pipeline.add_node(component=rerank, name="ReRanker", inputs=["JoinDocuments"])


In [ ]:
from haystack.schema import EvaluationResult, MultiLabel

# We can load evaluation labels from the document store
# We are also opting to filter out no_answer samples

k = 15

eval_labels = document_store.get_all_labels_aggregated(index=label_index, drop_negative_labels=True, drop_no_answers=True)

eval_result = pipeline.eval(labels=eval_labels, params={"SparseRetriever": {"top_k": k},
                             "DenseRetriever": {"top_k": k},
                             "JoinDocuments": {"top_k_join": 2*k},
                             "ReRanker": {"top_k": k},})
# eval_result = pipeline.eval(labels=eval_labels, params={"JoinDocuments": {"top_k_join": 2}})
# retriever_result = eval_result["JoinDocuments"]
retriever_result = eval_result["ReRanker"]
retriever_result.head()

,multilabel_id,query,filters,gold_answers,context,gold_contexts,gold_id_match,context_match,answer_match,gold_id_or_answer_match,...,rank,document_id,gold_document_ids,gold_documents_id_match,gold_contexts_similarity,gold_answers_match,type,node,eval_mode,index
0,de48d235de919a09b52cbf223c7c3dc7,ناقه الهی معجزه‌ای برای کدام قوم بود؟,b'null',"[ثمود, ثمود, ثمود, ثمود, ثمود, ثمود, ثمود, ثمو...",. چیزی ما را از انجام این معجزات باز نمی‌دارد،...,[و ما را از فرستادن نشانه‌ها- معجزات دلخواهشان...,1.0,1.0,1.0,1.0,...,1.0,1e29fad62daf27a1eaf12484521a296c-0,"[2414376d7e4a65fe30e2447dc969dd9b-0, 62150e38b...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[53.50877192982456, 64.61538461538461, 55.2083...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",document,ReRanker,integrated,0
1,de48d235de919a09b52cbf223c7c3dc7,ناقه الهی معجزه‌ای برای کدام قوم بود؟,b'null',"[ثمود, ثمود, ثمود, ثمود, ثمود, ثمود, ثمود, ثمو...",و هیچ چیز، ما را از فرستادن معجزات باز نداشت ج...,[و ما را از فرستادن نشانه‌ها- معجزات دلخواهشان...,1.0,1.0,1.0,1.0,...,2.0,b7a5c11fd0a40a8283d9b97d39706a18-0,"[2414376d7e4a65fe30e2447dc969dd9b-0, 62150e38b...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[57.62970259874284, 77.94871794871796, 65.1041...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",document,ReRanker,integrated,1
2,de48d235de919a09b52cbf223c7c3dc7,ناقه الهی معجزه‌ای برای کدام قوم بود؟,b'null',"[ثمود, ثمود, ثمود, ثمود, ثمود, ثمود, ثمود, ثمو...",و ما را از فرستادن آیات و معجزات جز تکذیب پیشی...,[و ما را از فرستادن نشانه‌ها- معجزات دلخواهشان...,1.0,1.0,1.0,1.0,...,3.0,e92cf08fabb4da5a7ea69193e8587c7e-0,"[2414376d7e4a65fe30e2447dc969dd9b-0, 62150e38b...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[55.118110236220474, 69.29133858267717, 100.0,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",document,ReRanker,integrated,2
3,de48d235de919a09b52cbf223c7c3dc7,ناقه الهی معجزه‌ای برای کدام قوم بود؟,b'null',"[ثمود, ثمود, ثمود, ثمود, ثمود, ثمود, ثمود, ثمو...",ما پیش از تو رسولانی فرستادیم؛ و برای آنها همس...,[و ما را از فرستادن نشانه‌ها- معجزات دلخواهشان...,0.0,0.0,0.0,0.0,...,4.0,ebc6e83cf5d7638021a48ce367e7b7b2-0,"[2414376d7e4a65fe30e2447dc969dd9b-0, 62150e38b...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[43.225806451612904, 46.298593879239036, 44.34...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",document,ReRanker,integrated,3
4,de48d235de919a09b52cbf223c7c3dc7,ناقه الهی معجزه‌ای برای کدام قوم بود؟,b'null',"[ثمود, ثمود, ثمود, ثمود, ثمود, ثمود, ثمود, ثمو...",و پیش از تو پیامبرانی فرستاده‌ایم و همسران و ف...,[و ما را از فرستادن نشانه‌ها- معجزات دلخواهشان...,0.0,0.0,0.0,0.0,...,5.0,7445270fd6a85fe7f563dbc2acb9ba5-0,"[2414376d7e4a65fe30e2447dc969dd9b-0, 62150e38b...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[43.24324324324324, 48.64864864864865, 45.9459...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",document,ReRanker,integrated,4


In [ ]:
eval_result.save("../")
saved_eval_result = EvaluationResult.load("../")
metrics = saved_eval_result.calculate_metrics()
print(f'ReRanker - Recall (single relevant document): {metrics["ReRanker"]["recall_single_hit"]}')
print(f'ReRanker - Recall (multiple relevant documents): {metrics["ReRanker"]["recall_multi_hit"]}')
print(f'ReRanker - Mean Reciprocal Rank: {metrics["ReRanker"]["mrr"]}')
print(f'ReRanker - Precision: {metrics["ReRanker"]["precision"]}')
print(f'ReRanker - Mean Average Precision: {metrics["ReRanker"]["map"]}')

ReRanker - Recall (single relevant document): 0.9548022598870056
ReRanker - Recall (multiple relevant documents): 0.9491051321559796
ReRanker - Mean Reciprocal Rank: 0.7010720603940943
ReRanker - Precision: 0.2753295668549906
ReRanker - Mean Average Precision: 0.6165663028276755


In [ ]:
print(f'SparseRetriever - Recall (single relevant document): {metrics["SparseRetriever"]["recall_single_hit"]}')
print(f'SparseRetriever - Recall (multiple relevant documents): {metrics["SparseRetriever"]["recall_multi_hit"]}')
print(f'SparseRetriever - Mean Reciprocal Rank: {metrics["SparseRetriever"]["mrr"]}')
print(f'SparseRetriever - Precision: {metrics["SparseRetriever"]["precision"]}')
print(f'SparseRetriever - Mean Average Precision: {metrics["SparseRetriever"]["map"]}')

SparseRetriever - Recall (single relevant document): 0.8531073446327684
SparseRetriever - Recall (multiple relevant documents): 0.8489984591679507
SparseRetriever - Mean Reciprocal Rank: 0.6292500500127618
SparseRetriever - Precision: 0.1902071563088512
SparseRetriever - Mean Average Precision: 0.5381234672312522


In [ ]:
print(f'DenseRetriever - Recall (single relevant document): {metrics["DenseRetriever"]["recall_single_hit"]}')
print(f'DenseRetriever - Recall (multiple relevant documents): {metrics["DenseRetriever"]["recall_multi_hit"]}')
print(f'DenseRetriever - Mean Reciprocal Rank: {metrics["DenseRetriever"]["mrr"]}')
print(f'DenseRetriever - Precision: {metrics["DenseRetriever"]["precision"]}')
print(f'DenseRetriever - Mean Average Precision: {metrics["DenseRetriever"]["map"]}')

DenseRetriever - Recall (single relevant document): 0.943502824858757
DenseRetriever - Recall (multiple relevant documents): 0.9383193078108333
DenseRetriever - Mean Reciprocal Rank: 0.7807654680536036
DenseRetriever - Precision: 0.29830508474576267
DenseRetriever - Mean Average Precision: 0.6769208634509566
